As it has been discussed in previous kernels, it is apparent each site has a different frequency per language.  For this, we will make predictions using the Ridge Regression model, as an example for future submissions for this dataset. 

We can use different linear models, or more complicate models, to make our predictions. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
key = pd.read_csv('../input/key_1.csv')
train = pd.read_csv('../input/train_1.csv')

In [ ]:
def get_language(page):
    import re
    
    res = re.search('[a-z][a-z].wikipedia.org',page)
    if res:
        return res[0][0:2]
    return 'na'

train['lang'] = train.Page.map(get_language)

Since each language has different frequencies, as seen and discussed in other kernels, lets train different models on each of the languages

In [ ]:
#splits training and testing sets into lists of sets per language
def lang_find( lang, df ):
    indices = np.where( df['lang'] == lang )[0]
    
    return df.loc[indices, :].drop('lang', axis = 1).copy()

def lang_df( df ):
    lang_list = df['lang'].unique()
        
    return {lang : lang_find(lang, df) for lang in lang_list}

This is the training set, divided by the languages.

In [ ]:
lang_train = lang_df( train )

We strip the dates in the string of each of the entries in the test set.

In [ ]:
#strips dates in testing data
def strip_date( page ):
    date_str = page.split('_')[-1:][0]
    
    return date_str

key['date'] = key.Page.map(strip_date)
key['lang'] = key.Page.map(get_language)

In [ ]:
lang_test = lang_df( key )

We use the ridge regression, since we can use different scoring metrics for our model selection per language.  We extract the year, month and day, and we try to predict the median visit per date to create a robust solution, not affected by any outliers.

In [ ]:
#feature engineering per language
def date_year( date ):
    return int(date.split('-')[0])

def date_month( date ):
    return int(date.split('-')[1])

def date_day( date ):
    return int(date.split('-')[2])

def lang_date_visit( df_lang ):
    visits = df_lang.drop('Page', axis = 1).median()
    
    df_visits = pd.DataFrame( columns = ['date', 'visit'] )
    
    df_visits['date'] = visits.index
    df_visits['visit'] = visits.values
    
    df_visits['year'] = df_visits.date.map(date_year)
    df_visits['month'] = df_visits.date.map(date_month)
    df_visits['day'] = df_visits.date.map(date_day)
     
    df_visits.drop('date', axis = 1, inplace = True)
    
    return df_visits

#evaluation metric
def smape(y_true, y_pred):
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)

#returns the model for the particular language
def lang_fit( df_lang ):
    
    from sklearn.metrics import make_scorer
    from sklearn.model_selection import TimeSeriesSplit
    from sklearn.linear_model import RidgeCV
    
    loss = make_scorer(smape, greater_is_better=False)
    
    df_visits = lang_date_visit( df_lang )
    
    model = RidgeCV(scoring = loss, cv = TimeSeriesSplit(10) )
    
    y = df_visits.pop('visit')
    
    print ('Fitting Model')
    
    return model.fit(df_visits, y)

#fits all the models per language
def fit( df_dict ):
    
    return { key : lang_fit( df_dict.get(key) ) for key in df_dict.keys() }

#makes prediction per language
def lang_predict( model, df_lang ):
    
    sub = pd.DataFrame( columns = ['Id', 'Prediction'])
    
    sub['Id'] = df_lang.pop('Id')
    
    df_lang['year'] = df_lang.date.map(date_year)
    df_lang['month'] = df_lang.date.map(date_month)
    df_lang['day'] = df_lang.date.map(date_day)
    
    df_lang.drop(['Page', 'date'], axis = 1, inplace = True)
    
    sub['Prediction'] = model.predict( df_lang )

    print ('Prediction Finished')
    
    return sub

#prediction for all languages
def predict( train_dict, test_dict ):
    
    model_dict = fit( train_dict )
    
    return { key : lang_predict( model_dict.get(key),
                                test_dict.get(key) ) for key in test_dict.keys() }

In [ ]:
for key in lang_train.keys():
    df_lang = lang_train.get(key)
    df_new = df_lang

In [ ]:
df_vis = df_lang.dropna().T.drop('Page', axis = 0).m(axis = 1).reset_index()

In [ ]:
df_vis.columns = ['date', 'visit']

In [ ]:
df_vis.plot()

In [ ]:
pred_dict = predict(lang_train, lang_test)

Now we append all the solutions together for submission.

In [ ]:
subs = pd.DataFrame( columns = ['Id', 'Prediction'] )

for key in pred_dict.keys():
    subs = subs.append( pred_dict.get(key) )

In [ ]:
subs.to_csv('../output/sub.csv', index = False)